In [1]:
import json
from preprocessor.preprocessor import Preprocessor
from model.model_keras import Model
from postprocessor.postprocessor import Postprocesser
from evaluator.evaluator import Evaluator
import numpy as np
import pandas as pd
import tensorflow as tf
import random

def set_seed(seed_value):
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    random.seed(seed_value)
set_seed(42)

with open('parameters.json', 'r') as file:
    params = json.load(file)

In [2]:
preprocessor = Preprocessor()
data = preprocessor.fetch_stock_data(params['stock_symbol'], params['start_date'], params['stop_date'])
for single_feature_params in params['features_params']:
    feature_type = single_feature_params["type"]
    data = preprocessor.add_feature(data, feature_type, **single_feature_params)
data, issues_detected = preprocessor.add_data_cleaner(data, 
    clean_type=params['data_cleaning']['clean_type'], strategy=params['data_cleaning']['strategy'])
X_train, y_train, X_test, y_test, train_dates, test_dates = \
    preprocessor.process_data(data, split_ratio=params['split_ratio'], target_col=params['target_col'], 
                            feature_cols=None, look_back=params['look_back'],
                            predict_steps=params['predict_steps'], 
                            train_slide_steps=params['train_slide_steps'], 
                            test_slide_steps=params['test_slide_steps'])
X_newest, x_newest_date = preprocessor.create_x_newest_data(data, params['look_back'])


[*********************100%***********************]  1 of 1 completed


c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\1201\project\preprocessor\preprocessor.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'].iloc[i:i + trend_days] = 0 # up trend
c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\1201\project\preprocessor\preprocessor.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'].iloc[i:i + trend_days] = 1 # down trend


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3430, 64, 19), (3430, 16, 2), (90, 64, 19), (90, 16, 2))

In [4]:
model_wrapper = Model()
model, history, y_preds, online_training_losses, online_training_acc = \
    model_wrapper.run(params['model_type'], params['look_back'], params['model_params'],
                        X_train, y_train, X_test, y_test)


{'conv_1_filter': 32, 'conv_1_kernel': 4, 'conv_1_l2': 0.00016475815766673417, 'dropout_1': 0.2, 'conv_2_filter': 96, 'conv_2_kernel': 2, 'dropout_2': 0.2, 'lstm_1_units': 128, 'lstm_1_l2': 0.0002788818914602332, 'dropout_3': 0.2, 'lstm_2_units': 64, 'dropout_4': 0.1, 'learning_rate': 0.001, 'look_back': 64, 'predict_steps': 16}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 64, 32)            2464      
                                                                 
 batch_normalization (BatchN  (None, 64, 32)           128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 64, 32)            0         
                                                                 
 dropout (Dropout)           (None, 64, 32)            

  0%|          | 0/90 [00:00<?, ?it/s]

1/1 [==============================] - 0s 483ms/step
1/1 - 0s - loss: 0.5187 - binary_accuracy: 0.7500 - 23ms/epoch - 23ms/step


  1%|          | 1/90 [00:00<01:00,  1.47it/s]

1/1 [==============================] - 0s 60ms/step
1/1 - 0s - loss: 0.3577 - binary_accuracy: 0.8750 - 20ms/epoch - 20ms/step


...

1/1 [==============================] - 0s 39ms/step
1/1 - 0s - loss: 0.3779 - binary_accuracy: 1.0000 - 18ms/epoch - 18ms/step


 99%|█████████▉| 89/90 [00:20<00:00,  4.42it/s]

1/1 [==============================] - 0s 30ms/step
1/1 - 0s - loss: 0.3706 - binary_accuracy: 1.0000 - 18ms/epoch - 18ms/step


100%|██████████| 90/90 [00:21<00:00,  4.25it/s]

Model saved as model/saves/model_20231202_220901.h5


In [5]:
postprocessor = Postprocesser()
test_trade_signals = postprocessor.process_signals(y_test, test_dates)
pred_trade_signals = postprocessor.process_signals(y_preds, test_dates)


In [6]:
evaluator = Evaluator()
evaluator.generate_report(model, y_test, y_preds, history, 
                        online_training_acc, online_training_losses,
                        data, pred_trade_signals, x_start=0, x_stop=300)

c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\1201\project\evaluator\evaluator.py:148: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buy_signals = trade_signals.loc[x_start:x_stop][(trade_signals['Signal'] == 'Buy') | (trade_signals['Signal'] == 'Buy (first)')]
c:\Users\e4903\Desktop\code\2023_hw\reverse_prediction\1201\project\evaluator\evaluator.py:153: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sell_signals = trade_signals.loc[x_start:x_stop][(trade_signals['Signal'] == 'Sell') | (trade_signals['Signal'] == 'Sell (first)')]
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
